# MongoDB Handling

After installing the MongoDB server in your machine, you can use this notebook for handling the initial processes with the database.

Specifically, in this step, we utilize Python's `pymongo` library to exploit its capabilities for MongoDB server interaction.

**Important Note: Be sure that the MongoDB server is up and running as a service in the background.**

For example, in macOS, to run MongoDB (i.e. the mongod process) as a service, run:

* `brew services start mongodb-community`

To stop a mongod running as a macOS service, use the following command as needed:

* `brew services stop mongodb-community`

To install MongoDB in your system, follow the instructions here:

* https://www.mongodb.com/docs/manual/administration/install-community/


**Note:** You can modify any of the processes below, however, you have to explain your thoughts.

In [1]:
# import library for various processes with the OS
import os

## Load configuration

In [2]:
# import library for yaml handling
import yaml

In [3]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## MongoDB database instantiation

The relevant information for the MongoDB client connection, the database name, and collection name is located in the configuration file.

```
# DB Connection with the uri (host)
client: "mongodb://localhost:27017/"

# db name
db: "aiot_course"

# db collection
col: "NAME YOUR COLLECTION"
```

In [4]:
# import library for hanlding the MongoDB client
import pymongo
# import library for retrieving datetime
from datetime import datetime

### Create the database

To create a database in MongoDB, start by creating a MongoClient object, then specify a connection URL with the correct ip address and the name of the database you want to create.

MongoDB will create the database if it does not exist, and make a connection to it.

In [5]:
client = pymongo.MongoClient(config["client"])

In [6]:
db = client[config["db"]]

### Instantiate the collection

To create a collection in MongoDB, use database object and specify the name of the collection you want to create.

MongoDB will create the collection if it does not exist.

In [7]:
col = db[config["col"]]

Initially, no collection will be shown in MongoDB before you enter the first document!

## Create the data collection

Uploading the gathered data to MongoDB collection. The data directory structure should be as follows:

```
.
└── data/
    ├── class_A/
    │   ├── data_A_01.csv
    │   ├── data_A_02.csv
    │   └── ..
    ├── class_B/
    │   ├── data_B_01.csv
    │   ├── data_B_02.csv
    │   └── .
    └── class ...
```

In [8]:
# import library for hanlding the csv data and transformations
import pandas as pd
import json

Get data path:

In [9]:
data_path = os.path.join(os.getcwd(), "data")
print(data_path)

c:\Users\spigg\Desktop\aiot_project\data


List all files in a path:

In [10]:
classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print(classes_folders_list)

['circle_left', 'circle_right', 'down', 'left', 'right', 'up', 'wrist_left', 'wrist_right']


In [11]:
# print files in folder
folder_path = os.path.join(data_path, classes_folders_list[0])
files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files_in_folder)

['acc_circle_left.csv', 'gyr_circle_left.csv']


Each document in the MongoDB database should have the following schema:

```json
{
  "data": {
    "acc_x": ["array", "of", "values"],
    "acc_y": ["array", "of", "values"],
    "acc_z": ["array", "of", "values"],
  },
  "label": "The label of the instance",
  "datetime": "MongoDB datetime object (it can be generated with the datetime.datetime.now() function"
}
```

Accordingly, if you are using gyroscope or both accelerometer and gyroscope, the following order and naming of the sensor keys should be defined:

* for gyroscope: `gyr_x`, `gyr_y`, `gyr_z` for the three axes
* for accelerometer and gyroscope: `acc_x`, `acc_y`, `acc_z`, `gyr_x`, `gyr_y`, `gyr_z` for the six axes

**Note: Be careful, the document is mandatory to have the aforementioned schema, in order to argue and proceed with the rest of the processes later on, in data engineering, plotting, etc.**

In [12]:
from utils import df_rebase
from utils import combine_data


## Provide the code to upload the data to MongoDB

In [13]:
for class_folder in classes_folders_list:
    folder_path = os.path.join(data_path, class_folder)
    print(f"Processing folder: {folder_path}")
    #print(f"Files in folder: {os.listdir(folder_path)}")
    acc_processed = False  # Flag 
    gyr_processed = False  # Flag 

    # Get the files in the current folder
    files_in_folder = os.listdir(folder_path)  
    print(f"Files in folder: {files_in_folder}")
    for file_name in files_in_folder: 
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file_name: {file_name}")
        print(f"Processing file_path: {file_path}" )
        if os.path.exists(file_path) and 'acc' in file_name:
            acc_file = pd.read_csv(file_path)
            print(acc_file)
            acc_processed = True  # Mark 'acc' as processed
        
        if os.path.exists(file_path) and 'gyr'in file_name:
             gyr_file = pd.read_csv(file_path)
             gyr_processed = True  # Mark 'acc' as processed
            
        if acc_processed and gyr_processed:
            combined_df = combine_data(acc_file, gyr_file)
            print(combined_df)
            df = df_rebase(combined_df,config['order'],config['rename'])
            print(df)
            acc_processed = False  # Flag 
            gyr_processed = False  # Flag 

            #Δημιουργεία του dictionary για το MongoDB
            data = df.to_dict(orient = 'list')
            label = class_folder
            timestamp = datetime.now()
            document = {"data": data, "label": label, "datetime": timestamp}
           # Insert document into MongoDB
            col.insert_one(document)


print('Documents inserted Successfully!')


Processing folder: c:\Users\spigg\Desktop\aiot_project\data\circle_left
Files in folder: ['acc_circle_left.csv', 'gyr_circle_left.csv']
Processing file_name: acc_circle_left.csv
Processing file_path: c:\Users\spigg\Desktop\aiot_project\data\circle_left\acc_circle_left.csv
          epoc (ms)        timestamp (+0300)  elapsed (s)  x-axis (g)  \
0     1727343608099  2024-09-26T12.40.08.099         0.00      -0.091   
1     1727343608119  2024-09-26T12.40.08.119         0.02      -0.061   
2     1727343608139  2024-09-26T12.40.08.139         0.04      -0.061   
3     1727343608159  2024-09-26T12.40.08.159         0.06      -0.054   
4     1727343608179  2024-09-26T12.40.08.179         0.08      -0.067   
...             ...                      ...          ...         ...   
3289  1727343674618  2024-09-26T12.41.14.618        66.52       0.061   
3290  1727343674638  2024-09-26T12.41.14.638        66.54       0.079   
3291  1727343674658  2024-09-26T12.41.14.658        66.56       0.055 